In [0]:
%pip install -U "google-adk[a2a]" "a2a-python[starlette]" "starlette" "uvicorn" "nest_asyncio"

  Using cached starlette-0.51.0-py3-none-any.whl.metadata (6.3 kB)
  Attempting uninstall: nest_asyncio
    Found existing installation: nest-asyncio 1.5.6
    Not uninstalling nest-asyncio at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-dc303545-402a-4d2a-9dd8-3a6bd3f5e1bb
    Can't uninstall 'nest-asyncio'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

### Build an ADK agent

In [0]:
import asyncio
import json
import os
import nest_asyncio

nest_asyncio.apply()

from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.server.apps import A2AStarletteApplication
from a2a.types import AgentCard, AgentSkill
from a2a.types import AgentCapabilities  # capabilities(streaming=...) in many examples

from google.adk import Runner
from google.adk.agents import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService

from google.adk.a2a.executor.a2a_agent_executor import A2aAgentExecutor, A2aAgentExecutorConfig



In [0]:
import datetime
from google.adk.agents import LlmAgent


import datetime

def check_availability(start_rfc3339: str, end_rfc3339: str, calendar_id: str = "primary") -> dict:
    # Stubbed for now
    return {
        "calendar_id": calendar_id,
        "start": start_rfc3339,
        "end": end_rfc3339,
        "is_free": True,
        "note": "Stubbed for Databricks testing. Replace with Google Calendar calls later.",
        "generated_at": datetime.datetime.now().isoformat(),
    }


### A2A Executor

In [0]:
import re

from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import TaskUpdater
from a2a.types import TaskState
from a2a.utils import new_agent_text_message, new_task

_RFC3339 = re.compile(r"\d{4}-\d{2}-\d{2}T[0-9:\.]+(?:Z|[+\-]\d{2}:\d{2})")

class CalendarAgentExecutor(AgentExecutor):
    async def execute(self, context: RequestContext, event_queue: EventQueue) -> None:
        # Get user text (version-tolerant)
        try:
            user_text = context.get_user_input()
        except Exception:
            user_text = ""
            if getattr(context, "message", None) and getattr(context.message, "parts", None):
                for p in context.message.parts:
                    if getattr(p, "kind", None) == "text":
                        user_text += getattr(p, "text", "") or ""

        # Ensure we have a task
        task = getattr(context, "current_task", None)
        if not task:
            task = new_task(context.message)  # create a task from the incoming message
            await event_queue.enqueue_event(task)

        updater = TaskUpdater(event_queue, task.id, task.context_id)

        # Parse timestamps
        times = _RFC3339.findall(user_text)
        if len(times) >= 2:
            start, end = times[0], times[1]
            result = check_availability(start, end, "primary")
            reply = f"is_free={result['is_free']} for {start} → {end} (calendar={result['calendar_id']})"
        else:
            reply = (
                "Please provide a time range in RFC3339, e.g. "
                "2026-01-12T10:00:00-05:00 to 2026-01-12T11:00:00-05:00"
            )

        await updater.update_status(
            TaskState.completed,
            new_agent_text_message(reply, task.context_id, task.id),
            final=True,
        )

    async def cancel(self, context: RequestContext, event_queue: EventQueue) -> None:
        raise NotImplementedError("cancel not supported in this notebook demo")

### Build A2A app

In [0]:
import os
from starlette.applications import Starlette
from starlette.requests import Request
from starlette.responses import PlainTextResponse, JSONResponse
from starlette.routing import Route

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import AgentCard, AgentCapabilities, AgentSkill

# Optional env checks (mirrors example style)
# You can remove this block if you don't want it.
if os.getenv("REQUIRE_API_KEY") == "TRUE" and not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY is required when REQUIRE_API_KEY=TRUE")

# Store auth callbacks in-memory (stub; replace with real token storage)
AUTH_CALLBACKS = {}

executor = CalendarAgentExecutor()

request_handler = DefaultRequestHandler(
    agent_executor=executor,
    task_store=InMemoryTaskStore(),
)

skill = AgentSkill(
    id="check_availability",
    name="Check Availability",
    description="Checks a user's availability for a time range (stubbed).",
    tags=["calendar"],
    examples=["Am I free from 2026-01-12T10:00:00-05:00 to 2026-01-12T11:00:00-05:00?"],
)

agent_card = AgentCard(
    name="Calendar Agent (Notebook)",
    description="A minimal A2A agent in Databricks with an /authenticate callback route.",
    url="http://localhost:0/",  # we'll patch the port after choosing it
    version="1.0.0",
    defaultInputModes=["text"],
    defaultOutputModes=["text"],
    capabilities=AgentCapabilities(streaming=True),
    skills=[skill],
)

a2a_app = A2AStarletteApplication(agent_card=agent_card, http_handler=request_handler)

# Base A2A routes:
# Different SDK builds expose either .routes() OR .build().routes
base_routes = None
if hasattr(a2a_app, "routes"):
    # many versions: routes() -> List[Route]
    try:
        base_routes = a2a_app.routes()
    except TypeError:
        pass

if base_routes is None:
    # fallback: build and grab .routes
    built = a2a_app.build()
    base_routes = list(getattr(built, "routes", []))

async def handle_auth(request: Request) -> PlainTextResponse:
    # This is OAuth-callback-shaped, like the example.
    # Typical query params: state, code, scope, error, etc.
    qp = dict(request.query_params)
    state = qp.get("state", "")
    AUTH_CALLBACKS[state or "NO_STATE"] = {
        "query_params": qp,
        "url": str(request.url),
        "received_at": datetime.datetime.now().isoformat(),
    }
    return PlainTextResponse("Authentication successful (captured callback params).")

async def health(_: Request) -> JSONResponse:
    return JSONResponse({"ok": True, "auth_callbacks": len(AUTH_CALLBACKS)})

extra_routes = [
    Route("/authenticate", endpoint=handle_auth, methods=["GET"]),
    Route("/health", endpoint=health, methods=["GET"]),
]

app = Starlette(routes=list(base_routes) + extra_routes)
app

In [0]:
from starlette.testclient import TestClient
from uuid import uuid4
import json

client = TestClient(app)

# 1) Check health
print("GET /health", client.get("/health").status_code, client.get("/health").json())

# 2) Hit authenticate callback
r = client.get("/authenticate?state=test-state&code=fake-code")
print("GET /authenticate", r.status_code, r.text)

# 3) Try A2A agent card endpoints (depends on SDK’s routing)
# We'll just probe a few likely ones and print status
for p in ["/.well-known/agent.json", "/.well-known/agent-card.json", "/"]:
    rr = client.get(p)
    print("GET", p, "->", rr.status_code)

# 4) Send a message via JSON-RPC to the root POST endpoint if your SDK uses it.
payload = {
    "jsonrpc": "2.0",
    "id": uuid4().hex,
    "method": "message/send",
    "params": {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": "Am I free from 2026-01-12T10:00:00-05:00 to 2026-01-12T11:00:00-05:00?"}],
            "messageId": uuid4().hex,
        }
    },
}

resp = client.post("/", json=payload)
print("\nPOST / message/send ->", resp.status_code)
print(resp.headers.get("content-type"))
print(resp.text[:1500])

INFO:py4j.clientserver:Received command c on object id p0
INFO:httpx:HTTP Request: GET http://testserver/health "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://testserver/health "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://testserver/authenticate?state=test-state&code=fake-code "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://testserver/.well-known/agent.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://testserver/.well-known/agent-card.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://testserver/ "HTTP/1.1 405 Method Not Allowed"
INFO:httpx:HTTP Request: POST http://testserver/ "HTTP/1.1 200 OK"


GET /health 200 {'ok': True, 'auth_callbacks': 0}
GET /authenticate 200 Authentication successful (captured callback params).
GET /.well-known/agent.json -> 200
GET /.well-known/agent-card.json -> 200
GET / -> 405

POST / message/send -> 200
application/json
{"id":"dc70b3bff99245b0a043c9856dea3960","jsonrpc":"2.0","result":{"contextId":"f356cbd7-2904-41df-bfa5-1c61c0a5f2c0","history":[{"contextId":"f356cbd7-2904-41df-bfa5-1c61c0a5f2c0","kind":"message","messageId":"5ceb66789c7549cdac6e2adf9b11a141","parts":[{"kind":"text","text":"Am I free from 2026-01-12T10:00:00-05:00 to 2026-01-12T11:00:00-05:00?"}],"role":"user","taskId":"70d97eb8-b5ff-49bf-a6f1-5333550cafe7"}],"id":"70d97eb8-b5ff-49bf-a6f1-5333550cafe7","kind":"task","status":{"message":{"contextId":"f356cbd7-2904-41df-bfa5-1c61c0a5f2c0","kind":"message","messageId":"5d2385e2-0ecd-4c4d-a5e5-10a73aec2ab3","parts":[{"kind":"text","text":"is_free=True for 2026-01-12T10:00:00-05:00 → 2026-01-12T11:00:00-05:00 (calendar=primary)"}],"ro

In [0]:
import socket
import uvicorn

def pick_free_port() -> int:
    s = socket.socket()
    s.bind(("0.0.0.0", 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = pick_free_port()

# Patch the agent card URL to include the real port
agent_card.url = f"http://localhost:{port}/"

config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
server = uvicorn.Server(config)

print("Starting server on port:", port)
await server.serve()

INFO:     Started server process [66058]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:35235 (Press CTRL+C to quit)


Starting server on port: 35235


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
server.should_exit = True
print("Shutdown requested.")


Shutdown requested.


### Agent Card

In [0]:
WEATHERBOT_CARD = {
  "name": "WeatherBot",
  "description": "Provides accurate weather forecasts and historical data.",
  "url": "http://localhost:0/",  # we'll patch port later
  "version": "1.0.0",
  "capabilities": {
    "streaming": True,
    "pushNotifications": False,
    "stateTransitionHistory": True
  },
  "authentication": {
    "schemes": ["apiKey"]
  },
  "defaultInputModes": ["text"],
  "defaultOutputModes": ["text"],
  "skills": [
    {
      "id": "get_current_weather",
      "name": "Get Current Weather",
      "description": "Retrieve real-time weather for any location.",
      "inputModes": ["text"],
      "outputModes": ["text"],
      "examples": ["What's the weather in Paris?", "Current conditions in Tokyo"],
      "tags": ["weather", "current", "real-time"]
    },
    {
      "id": "get_forecast",
      "name": "Get Forecast",
      "description": "Get 5-day weather predictions.",
      "inputModes": ["text"],
      "outputModes": ["text"],
      "examples": ["5-day forecast for New York", "Will it rain in London this weekend?"],
      "tags": ["weather", "forecast", "prediction"]
    }
  ]
}

WEATHERBOT_CARD

{'name': 'WeatherBot',
 'description': 'Provides accurate weather forecasts and historical data.',
 'url': 'http://localhost:0/',
 'version': '1.0.0',
 'capabilities': {'streaming': True,
  'pushNotifications': False,
  'stateTransitionHistory': True},
 'authentication': {'schemes': ['apiKey']},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'skills': [{'id': 'get_current_weather',
   'name': 'Get Current Weather',
   'description': 'Retrieve real-time weather for any location.',
   'inputModes': ['text'],
   'outputModes': ['text'],
   'examples': ["What's the weather in Paris?", 'Current conditions in Tokyo'],
   'tags': ['weather', 'current', 'real-time']},
  {'id': 'get_forecast',
   'name': 'Get Forecast',
   'description': 'Get 5-day weather predictions.',
   'inputModes': ['text'],
   'outputModes': ['text'],
   'examples': ['5-day forecast for New York',
    'Will it rain in London this weekend?'],
   'tags': ['weather', 'forecast', 'prediction']}]}

In [0]:
import datetime

def get_current_weather(location: str) -> dict:
    return {
        "location": location,
        "temp_c": 20,
        "condition": "Partly cloudy",
        "observed_at": datetime.datetime.now().isoformat(),
        "note": "Stubbed response"
    }

def get_forecast(location: str, days: int = 5) -> dict:
    return {
        "location": location,
        "days": days,
        "forecast": [{"day": i+1, "condition": "Sunny", "high_c": 22, "low_c": 15} for i in range(days)],
        "generated_at": datetime.datetime.now().isoformat(),
        "note": "Stubbed response"
    }


### A2A Executor

In [0]:
import re

from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import TaskUpdater
from a2a.types import TaskState
from a2a.utils import new_agent_text_message, new_task

def _extract_text(context: RequestContext) -> str:
    try:
        return context.get_user_input()
    except Exception:
        # fallback
        txt = ""
        msg = getattr(context, "message", None)
        if msg and getattr(msg, "parts", None):
            for p in msg.parts:
                if getattr(p, "kind", None) == "text":
                    txt += getattr(p, "text", "") or ""
        return txt

# Very small location extractor (stub)
# Example prompts:
# - "What's the weather in Paris?"
# - "5-day forecast for New York"
_LOC = re.compile(r"(?:in|for)\s+([A-Za-z][A-Za-z\s\-]+)$", re.IGNORECASE)

class WeatherBotExecutor(AgentExecutor):
    async def execute(self, context: RequestContext, event_queue: EventQueue) -> None:
        user_text = _extract_text(context).strip()

        task = getattr(context, "current_task", None)
        if not task:
            task = new_task(context.message)
            await event_queue.enqueue_event(task)

        updater = TaskUpdater(event_queue, task.id, task.context_id)

        # Decide which "skill" based on user text
        text_lower = user_text.lower()

        # default location parsing
        m = _LOC.search(user_text)
        location = (m.group(1).strip() if m else "New York")

        if "forecast" in text_lower or "5-day" in text_lower or "weekend" in text_lower:
            data = get_forecast(location, days=5)
            reply = f"Forecast for {data['location']} (next {data['days']} days): " + ", ".join(
                f"day{i+1}:{d['condition']} {d['low_c']}–{d['high_c']}°C"
                for i, d in enumerate(data["forecast"])
            )
        else:
            data = get_current_weather(location)
            reply = f"Current weather in {data['location']}: {data['condition']}, {data['temp_c']}°C."

        await updater.update_status(
            TaskState.completed,
            new_agent_text_message(reply, task.context_id, task.id),
            final=True,
        )

    async def cancel(self, context: RequestContext, event_queue: EventQueue) -> None:
        raise NotImplementedError("cancel not supported")


### Build AgentCard from JSON

In [0]:
from a2a.types import AgentCard, AgentCapabilities, AgentSkill
import copy

def build_agent_card_from_json(card_json: dict) -> AgentCard:
    card_json = copy.deepcopy(card_json)

    # Capabilities (typed)
    caps_in = card_json.get("capabilities", {}) or {}
    capabilities = AgentCapabilities(streaming=bool(caps_in.get("streaming", False)))

    # Skills (typed)
    skills = []
    for s in card_json.get("skills", []) or []:
        skills.append(
            AgentSkill(
                id=s["id"],
                name=s.get("name", s["id"]),
                description=s.get("description", ""),
                tags=s.get("tags", []),
                examples=s.get("examples", []),
            )
        )

    # Put anything not representable in typed fields into extensions
    extensions = {}
    for k in ["authentication", "capabilities"]:
        if k in card_json:
            extensions[k] = card_json[k]

    # Include skill I/O modes in extensions too (AgentSkill type may not carry them)
    extensions["skills_raw"] = card_json.get("skills", [])

    return AgentCard(
        name=card_json["name"],
        description=card_json.get("description", ""),
        url=card_json.get("url", "http://localhost:0/"),
        version=card_json.get("version", "1.0.0"),
        defaultInputModes=card_json.get("defaultInputModes", ["text"]),
        defaultOutputModes=card_json.get("defaultOutputModes", ["text"]),
        capabilities=capabilities,
        skills=skills,
        # Not all SDK versions support `extensions` as a typed field;
        # if yours doesn't, we'll attach it later when serving.
    ), extensions

agent_card, agent_card_extensions = build_agent_card_from_json(WEATHERBOT_CARD)
agent_card, agent_card_extensions.keys()


(AgentCard(additional_interfaces=None, capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True), default_input_modes=['text'], default_output_modes=['text'], description='Provides accurate weather forecasts and historical data.', documentation_url=None, icon_url=None, name='WeatherBot', preferred_transport='JSONRPC', protocol_version='0.3.0', provider=None, security=None, security_schemes=None, signatures=None, skills=[AgentSkill(description='Retrieve real-time weather for any location.', examples=["What's the weather in Paris?", 'Current conditions in Tokyo'], id='get_current_weather', input_modes=None, name='Get Current Weather', output_modes=None, security=None, tags=['weather', 'current', 'real-time']), AgentSkill(description='Get 5-day weather predictions.', examples=['5-day forecast for New York', 'Will it rain in London this weekend?'], id='get_forecast', input_modes=None, name='Get Forecast', output_modes=None, secu

In [0]:
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore

from starlette.applications import Starlette
from starlette.routing import Route
from starlette.responses import JSONResponse
from starlette.requests import Request

executor = WeatherBotExecutor()

handler = DefaultRequestHandler(
    agent_executor=executor,
    task_store=InMemoryTaskStore(),
)

server = A2AStarletteApplication(agent_card=agent_card, http_handler=handler)

# Base A2A routes from the SDK
base_app = server.build()
base_routes = list(getattr(base_app, "routes", []))

# We'll serve the card JSON exactly like your example
async def agent_json(_: Request):
    # patch url later when we know port; for now keep placeholder
    card = copy.deepcopy(WEATHERBOT_CARD)
    return JSONResponse(card)

app = Starlette(routes=base_routes + [Route("/.well-known/agent.json", agent_json, methods=["GET"])])

app

In [0]:
from starlette.testclient import TestClient
from uuid import uuid4
import json

client = TestClient(app)

print("GET /.well-known/agent.json ->", client.get("/.well-known/agent.json").status_code)
print(json.dumps(client.get("/.well-known/agent.json").json(), indent=2)[:1200])

payload = {
    "jsonrpc": "2.0",
    "id": uuid4().hex,
    "method": "message/send",
    "params": {
        "message": {
            "role": "user",
            "parts": [{"kind": "text", "text": "5-day forecast for Paris"}],
            "messageId": uuid4().hex,
        }
    },
}

resp = client.post("/", json=payload)
print("\nPOST / message/send ->", resp.status_code)
print(resp.text[:1500])


INFO:httpx:HTTP Request: GET http://testserver/.well-known/agent.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://testserver/.well-known/agent.json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://testserver/ "HTTP/1.1 200 OK"


GET /.well-known/agent.json -> 200
{
  "capabilities": {
    "streaming": true
  },
  "defaultInputModes": [
    "text"
  ],
  "defaultOutputModes": [
    "text"
  ],
  "description": "Provides accurate weather forecasts and historical data.",
  "name": "WeatherBot",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "Retrieve real-time weather for any location.",
      "examples": [
        "What's the weather in Paris?",
        "Current conditions in Tokyo"
      ],
      "id": "get_current_weather",
      "name": "Get Current Weather",
      "tags": [
        "weather",
        "current",
        "real-time"
      ]
    },
    {
      "description": "Get 5-day weather predictions.",
      "examples": [
        "5-day forecast for New York",
        "Will it rain in London this weekend?"
      ],
      "id": "get_forecast",
      "name": "Get Forecast",
      "tags": [
        "weather",
        "forecast",
        "prediction"


In [0]:
import socket
import uvicorn

def pick_free_port() -> int:
    s = socket.socket()
    s.bind(("0.0.0.0", 0))
    p = s.getsockname()[1]
    s.close()
    return p

port = pick_free_port()

# Patch WeatherBot card URL dynamically (so discovery is accurate)
WEATHERBOT_CARD["url"] = f"http://localhost:{port}/"

config = uvicorn.Config(app, host="0.0.0.0", port=port, log_level="info")
server = uvicorn.Server(config)

print("Starting WeatherBot on port:", port)
await server.serve()


INFO:     Started server process [66058]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:39101 (Press CTRL+C to quit)


Starting WeatherBot on port: 39101


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
server.should_exit = True
print("Shutdown requested.")


Shutdown requested.
